1. Extract multiword expression candidates. 
	1. Using the part-of-speech tags we extract multiword expression candidates, consisting of sequences of zero or more adjectives (ADJ followed by nouns (NOUN) or proper nouns (PROPNs) sequences.
	2. To generate training data for sequence tagging use sentence encoder like 
		a. EmbedRank (Bennani- Smires et al., 2018
		b. Key2Vec (Mahata et al., 2018)
    3. We implemented our Unsupervised Annotator using the POS tagger of SpaCy (Honnibal et al., 2020).

In [1]:
import re
!! pip install spacy 

['Requirement already satisfied: spacy in c:\\users\\qyzyr\\onedrive\\documents\\project_nlp\\unsupervised_annotator1\\venv\\lib\\site-packages (3.6.1)',
 'Requirement already satisfied: spacy-legacy<3.1.0,>=3.0.11 in c:\\users\\qyzyr\\onedrive\\documents\\project_nlp\\unsupervised_annotator1\\venv\\lib\\site-packages (from spacy) (3.0.12)',
 'Requirement already satisfied: spacy-loggers<2.0.0,>=1.0.0 in c:\\users\\qyzyr\\onedrive\\documents\\project_nlp\\unsupervised_annotator1\\venv\\lib\\site-packages (from spacy) (1.0.4)',
 'Requirement already satisfied: murmurhash<1.1.0,>=0.28.0 in c:\\users\\qyzyr\\onedrive\\documents\\project_nlp\\unsupervised_annotator1\\venv\\lib\\site-packages (from spacy) (1.0.9)',
 'Requirement already satisfied: cymem<2.1.0,>=2.0.2 in c:\\users\\qyzyr\\onedrive\\documents\\project_nlp\\unsupervised_annotator1\\venv\\lib\\site-packages (from spacy) (2.0.7)',
 'Requirement already satisfied: preshed<3.1.0,>=3.0.2 in c:\\users\\qyzyr\\onedrive\\documents\\pr

In [2]:
!! python -m spacy download en_core_web_sm


['Collecting en-core-web-sm==3.6.0',
 '  Downloading https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.6.0/en_core_web_sm-3.6.0-py3-none-any.whl (12.8 MB)',
 '     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--',
 '     --------------------------------------- 0.0/12.8 MB 653.6 kB/s eta 0:00:20',
 '     --------------------------------------- 0.0/12.8 MB 653.6 kB/s eta 0:00:20',
 '     --------------------------------------- 0.1/12.8 MB 469.7 kB/s eta 0:00:28',
 '     --------------------------------------- 0.1/12.8 MB 469.7 kB/s eta 0:00:28',
 '     --------------------------------------- 0.1/12.8 MB 469.7 kB/s eta 0:00:28',
 '      -------------------------------------- 0.2/12.8 MB 654.4 kB/s eta 0:00:20',
 '      -------------------------------------- 0.2/12.8 MB 654.4 kB/s eta 0:00:20',
 '      -------------------------------------- 0.2/12.8 MB 654.4 kB/s eta 0:00:20',
 '      -------------------------------------- 0.2/12.8 MB 554.9 kB/

In [3]:
import spacy
from spacy.tokenizer import Tokenizer
from spacy.util import compile_infix_regex
# Load the language model
nlp = spacy.load('en_core_web_sm')



In [4]:

# Define your text
with open('abstracts.txt') as file:
    text=file.read()


In [5]:
# text = "I love New York and San Francisco. Los Angeles is another great city."


In [6]:
from collections import namedtuple
CandidateMWE = namedtuple('CandidateMWE',['text','head', 'sentence'])

In [7]:
# Process the text with spaCy
doc = nlp(text)

# Extract MWEs (noun phrases) from the text
mwe_list = []
single_noun_list=[]
candidate_list=[]
for sent in doc.sents:
    for chunk in doc.noun_chunks:
        mwe_list.append(chunk.text)
        
        if len(chunk.text.split()) == 1:
            single_noun_list.append((chunk.text, chunk.lemma_))
        if len(chunk.text.split()) > 1:
            noun_appeared=False
            is_candidate=True
            cleared_candidate=''
            for word in chunk:
                #IGNORING
                if word.pos_ in ['PUNCT', 'DET']:
                    continue
                elif word.pos_ not in ['ADJ', 'PROPN', 'NOUN']:
                    is_candidate=False
                    print(f'{chunk.text} is not candidate 1 {word.text} -- {word.pos_}')
                    break
                elif word.pos_ in ['PROPN', 'NOUN']:
                    noun_appeared=True
                elif not(not noun_appeared and word.pos_=='ADJ'):
                    is_candidate=False
                    print(f'{chunk.text} is not candidate 2 {word.text} -- {word.pos_}, {noun_appeared}')
                    break
                cleared_candidate+=' '+word.text
            if is_candidate:
                candidate_list.append(CandidateMWE(cleared_candidate, chunk.root.text, sent.text))
            else:
                single_noun_list +=[(word.text, word.lemma_) for word in chunk if word.pos_ in ['NOUN', 'PROPN']]
                    
            
# print(mwe_list)


Supervised learning models is not candidate 1 Supervised -- VERB
i.e. tasks is not candidate 1 i.e. -- X
little labeled data is not candidate 1 labeled -- VERB
One prominent approach is not candidate 1 One -- NUM
unsupervised pre-trained neural models is not candidate 1 trained -- VERB
these two approaches is not candidate 1 two -- NUM
two novel methods is not candidate 1 two -- NUM
a more complex context-aware method is not candidate 1 more -- ADV
two English text classification datasets is not candidate 1 two -- NUM
our investigation is not candidate 1 our -- PRON
train-set sizes is not candidate 1 set -- VERB
only a few dozen training instances is not candidate 1 only -- ADV
more powerful dialogue modeling capabilities is not candidate 1 more -- ADV
a pre-trained model is not candidate 1 trained -- VERB
three types is not candidate 1 three -- NUM
two dialogue summarization datasets is not candidate 1 two -- NUM
-trained and non pre-trained models is not candidate 1 trained -- VERB
o

In [8]:
!!pip install sentence-transformers

['Requirement already satisfied: sentence-transformers in c:\\users\\qyzyr\\onedrive\\documents\\project_nlp\\unsupervised_annotator1\\venv\\lib\\site-packages (2.2.2)',
 'Requirement already satisfied: transformers<5.0.0,>=4.6.0 in c:\\users\\qyzyr\\onedrive\\documents\\project_nlp\\unsupervised_annotator1\\venv\\lib\\site-packages (from sentence-transformers) (4.33.0)',
 'Requirement already satisfied: tqdm in c:\\users\\qyzyr\\onedrive\\documents\\project_nlp\\unsupervised_annotator1\\venv\\lib\\site-packages (from sentence-transformers) (4.66.1)',
 'Requirement already satisfied: torch>=1.6.0 in c:\\users\\qyzyr\\onedrive\\documents\\project_nlp\\unsupervised_annotator1\\venv\\lib\\site-packages (from sentence-transformers) (2.0.1)',
 'Requirement already satisfied: torchvision in c:\\users\\qyzyr\\onedrive\\documents\\project_nlp\\unsupervised_annotator1\\venv\\lib\\site-packages (from sentence-transformers) (0.15.2)',
 'Requirement already satisfied: numpy in c:\\users\\qyzyr\\on

In [9]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [10]:
from sentence_transformers import SentenceTransformer, util    
from typing import List

In [11]:
def dist(wi:str, wj:str)->float:
    return util.pytorch_cos_sim(
        model.encode(wi, convert_to_tensor=True),
        model.encode(wj, convert_to_tensor=True)
    )


def calculate_topic_score(multiword_expression:str, sentence:str)->float:
    """
    Calculate the topic score between a multiword expression and a sentence.

    Args:
        multiword_expression (str): The multiword expression.
        sentence (str): The sentence containing the expression.

    Returns:
        float: The topic score (cosine similarity) between the two embeddings.
    """
    # Load the distilbert-base-nli-mean-tokens model

    # Encode the multiword expression and sentence into embeddings
    expression_embedding = model.encode(multiword_expression, convert_to_tensor=True)
    sentence_embedding = model.encode(sentence, convert_to_tensor=True)

    # Calculate cosine similarity between the two embeddings
    similarity_score = util.pytorch_cos_sim(expression_embedding, sentence_embedding)

    # Extract the cosine similarity value from the tensor
    topic_score = similarity_score[0].item()

    return topic_score

def calculate_specificity_score(mw:str, w:List[str])->float:
    """
    Calculate the specificity score (SP) between a multiword expression (mw) and a list of words/multiword expressions (w).

    Args:
        mw (str): The multiword expression.
        w (list of str): The list of words/multiword expressions in the context.

    Returns:
        float: The specificity score (SP).
    """
    # Load the distilbert-base-nli-mean-tokens model
    # Calculate distances between mw and each word/phrase in w
    distances = [dist(mw, wi) for wi in w if wi != mw]

    # Calculate the mean of the distances
    specificity_score = sum(distances) / len(w)

    return specificity_score


In [12]:
TSP = 0.05
Ttopic = 0.1

In [13]:
term_mws=[]

In [ ]:
for candidate in candidate_list:
    topic_score=calculate_topic_score(candidate.text, candidate.sentence)
    sp_score=calculate_specificity_score(candidate.text, mwe_list)
    if topic_score>Ttopic and sp_score>TSP:
        term_mws.append(candidate)
        print(candidate.text, topic_score, sp_score)
    

 low shot tasks 0.5353471636772156 tensor([[0.5793]])


3. Upgrade single nouns according to morphological features.
	1. At this stage, we could have nouns that are not part of any multiword expressions, but still relevant.
	2. Check if the lemma of the noun is the same as any of the heads of the multiword expressions.
		a. Yes: we upgrade the noun to term 
		b. No: segment the word using a subword-unit segmentation and a vocabulary trained over a large general purpose corpus.
we use the vocabulary of the BERT-base model from HuggingFace (Wolf et al., 2020) and the corresponding tokenizer.

In [ ]:
from transformers import BertTokenizer

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
term_nouns=[]
subtoken_threshold=4
for noun, lemma in single_noun_list:
    #Check if the lemma of the noun is the same as any of the heads of the multiword expressions.
    is_term=False
    lemma_is_head=False
    for term_mw in term_mws:
        if term_mw.head==lemma:
            is_term=True
            break
    if is_term:
        term_nouns.append(noun)
        continue
    #segment the word using a subword-unit segmentation and a vocabulary trained over a large general purpose corpus.
    subtokens=tokenizer.tokenize(noun)
    if len(subtokens)>subtoken_threshold:
        term_nouns.append(noun)

In [ ]:
terms=[term_mw.text for term_mw in term_mws]+term_nouns
with open('out.txt', 'w') as out_file:
    print(terms)